<center> <H1> FRAPtrackAnalyser Classes </H1> </center>

    
            *Code written by Emilie Cuillery and Timo Rey. Laboratory of Experimental Biophysics, EPFL*
                                            *Created between 2018 - 2020*
#### Aims:
    Streamline FRAPtA_Analysis scripts.
#### Use of script:
    Save in same directory as FRAPtA_Analysis scripts.

In [1]:
#load libraries:

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab
import numpy as np
from scipy.optimize import curve_fit
from scipy import stats
import seaborn as sns

In [2]:
# Define exponential types:
# single exponential
def single_exp(x, a, b):
    return a*(1-np.exp(-b*x))

# double exponential
def double_exp(x, a, b, c, d):
    return a*(1-np.exp(-b*x))+c*(1-np.exp(-d*x))

# set default:
single = True

**Definition**

    each experimental class contains the following defining values:

    Condition_name = FRAPcondition(
    r'Condition_name',                                               #Name
    r'Path',                                                         #Path
    'Subset',                                                        #Subset
    Nb_Frames,                                                       #Nb_Frames
    Prebleach_Frames,                                                #Preableach_Steps
    SecondsPerFrame                                                  #after bleach)

# Class "FRAPcondition"

In [3]:
class FRAPcondition:
    """ Create a new condition, defined by several parameters """

                        #{0}  #{1}  #{2}     #{3}       #{4} 
    def __init__(self, Name, Path, Subset, Nb_Frames, Prebleach_Frames, SecondsPerFrame):

        self.Name = Name
        self.Path = Path
        self.Subset = Subset
        self.Nb_Frames = Nb_Frames
        self.Prebleach_Frames = Prebleach_Frames
        self.SecondsPerFrame = SecondsPerFrame

        self.txt_List = None
        self.Data = pd.DataFrame()
        self.Mean = []
        self.Std = []
        self.Mobile_Fraction_ave = None #Mobile fraction calculated by averaging all the data
        self.Mobile_Fraction = []
        self.Half_Time_ave = None #Half Time calculated by averaging all the data
        self.Half_Time = []
    
        print("Condition called '{0}' was successfully created".format(self.Name))   


    def __str__(self): #can call this with print(self)
        return """Condition:
 Name: {0} \n Path: {1} \n Subset: {2} \n Number of frames analyzed: {3} \n Number of pre-bleach frames: {4}
 Seconds per frame: {5}s""".format(self.Name, self.Path, self.Subset, self.Nb_Frames, self.Prebleach_Frames, self.SecondsPerFrame)
    
    def Extract_Data(self):
        """ Find .txt files and read them to extract data. Also compute normalisedFS data """
        
        print("Find data for ", self.Name)
        
        #PART 1 - Get txt_list                                                                      
        self.txt_List = sorted(Path(self.Path).glob('*' + self.Subset + '*.txt'))        
        # Get Data
        Parser = 0
        print("Extract data on",self.Nb_Frames, "frames and for",len(self.txt_List) ,"files")
        
        while Parser < len(self.txt_List):           
            #Read file and get Raw_Data
            with open(str(self.txt_List[Parser]), 'r') as file:                         
                Raw_Data= pd.read_csv(file, sep = '\t')                            
                Raw_Data.columns = ['FRAME','TIME', 'FRAP', 'REF', 'BACK']
                if Parser==0:
                    self.Data['FRAME'] = pd.Series(Raw_Data['FRAME'].iloc[:self.Nb_Frames])
                    self.Data['TIME'] = pd.Series(Raw_Data['TIME'].iloc[:self.Nb_Frames])
            
        #PART 2 - Normalise FRAP Intensities
            Prebleach = Raw_Data[:int(self.Prebleach_Frames)]                    #Get pre-bleach data                                                   
           
            CorrPreREF = Prebleach['REF']-Prebleach['BACK']                      #Compute parameters
            Iref_pre = CorrPreREF.mean()
            CorrPreFRAP = Prebleach['FRAP']-Prebleach['BACK']
            Ifrap_pre = CorrPreFRAP.mean()
            
            Ifrap = Raw_Data['FRAP']
            Iref = Raw_Data['REF']
            Iback = Raw_Data['BACK']
            
            Inorm = (Iref_pre / (Iref - Iback)) * ((Ifrap - Iback) / Ifrap_pre)  #Double normalisation
                    #see FRAPAnalyser here: http://actinsim.uni.lu/eng/Downloads for more insights 
    
            Inorm_pre = 0                                                        #Full scaling starts
            for i in range(self.Prebleach_Frames):
                Inorm_pre = Inorm_pre + Inorm[i]
            Inorm_pre = Inorm_pre / self.Prebleach_Frames
            Inorm_FULLSCALE = (Inorm - Inorm[self.Prebleach_Frames])/(Inorm_pre - Inorm[self.Prebleach_Frames])
    
            Raw_Data['NormalizedFS'] = pd.Series(Inorm_FULLSCALE)
        
            #Add df to DATA
            self.Data['#'+str(Parser)+'_FRAP_'+str(self.Subset)] = pd.Series(Raw_Data['FRAP'].iloc[:self.Nb_Frames])
            self.Data['#'+str(Parser)+'_REF_'+str(self.Subset)] = pd.Series(Raw_Data['REF'].iloc[:self.Nb_Frames])
            self.Data['#'+str(Parser)+'_BACK_'+str(self.Subset)] = pd.Series(Raw_Data['BACK'].iloc[:self.Nb_Frames])
            self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)] = pd.Series(Raw_Data['NormalizedFS'].iloc[:self.Nb_Frames])
            
            Parser += 1
        print("Raw data extracted and normalised.")

        
    def Mean_and_Std(self):
        """ Compute Mean and Standard deviation """            
        
        tmp = [0]*len(self.txt_List) # creates array with y values for each time point (length txtList)

        for i in range(0,self.Nb_Frames):
            for Parser in range(0,len(self.txt_List)):
                tmp[Parser] = self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)][i]
            self.Mean.append(np.mean(tmp))
            self.Std.append(np.std(tmp))
        
        print("Mean and Sandard deviation were successfully computed.")

        
    def FitData(self):
        """ Fit all normalised recovery curves and plot fit with std """
    
        for Parser in range(0,len(self.txt_List)):
            x=self.Data['TIME'][0:self.Nb_Frames-int(self.Prebleach_Frames)]-1
            y=self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)][int(self.Prebleach_Frames):self.Nb_Frames]
            Residues = np.zeros(len(y))
        
            popt, pcov = curve_fit(single_exp, x, y)
    
            #Store values
            self.Mobile_Fraction.append(popt[0])
            self.Half_Time.append(-1*np.log(0.5)/(popt[1]/self.SecondsPerFrame))
            
        print(str(len(self.txt_List)) + " data was fit as well as MF & HT computed.")


    def PlotFitWerror(self, outDir=None, single=False, color1 = 'black', color2 = 'forestgreen', color3  = 'peru', 
                     xAxis = r'time [s]', yAxis = 'norm. intensity'):
        """ Fit all normalised recovery curves and plot fit with std """
        
        #choosing single or double exponential fit as a model:
        if single == True:
            exponential = single_exp
            print("you are fitting with a single exponential")
        elif single == False:
            exponential = double_exp
            print("you are fitting with a double exponential")
        
        #Redo the fit:   
        y_mean_plus = []                                                             #upper bound
        for i in range(0,self.Nb_Frames):
            y_mean_plus.append(self.Mean[i] + self.Std[i])  
        y_mean_minus = []                                                            #lower bound 
        for i in range(0,self.Nb_Frames):
            y_mean_minus.append(self.Mean[i] - self.Std[i])

        x=self.Data['TIME'][0:self.Nb_Frames-int(self.Prebleach_Frames)]-1
        y=self.Mean[int(self.Prebleach_Frames):self.Nb_Frames]
        popt, pcov = curve_fit(exponential, x, y, p0=None)
        
        xx = np.linspace(0,self.Nb_Frames,1000)
        yy = exponential(xx, *popt)

        xxx = (self.Data['TIME'] - self.Data['TIME'][self.Prebleach_Frames])        
        
        #output the fit:
        ax0 = plt.subplot2grid((3, 3), (0, 0), colspan=3, rowspan=4)                         #define geometry of the plot

        ax0.plot(xxx, self.Mean[:], 'b-', color = color1, label = 'Averaged FRAP recovery')  #Mean
        ax0.plot(xxx, y_mean_plus[:], "y--", color = color2)                                 #std+
        ax0.plot(xxx, y_mean_minus[:], "y--", color = color2)                                #std-
        ax0.plot(xx, yy,'k-', color = color3)                                                #fit
        ax0.set_xlim(xxx[0], xxx[self.Nb_Frames-1])
        ax0.set_ylim([0, 1.2])
        ax0.set_title(self.Name)
        ax0.set_xlabel(xAxis)
        ax0.set_ylabel(yAxis)
        
        if outDir != None:
            plt.savefig(outDir + self.Name + ".svg")
            print("This plot was saved at: " + outDir)
        else:
            print("This plot was not saved")
 
    def PlotIndFit(self):
        """ Recalculate fits and plot individually """
    
        for Parser in range(0,len(self.txt_List)):
            x=(self.Data['FRAME'][0:self.Nb_Frames-int(self.Prebleach_Frames)]-1)*self.SecondsPerFrame
            y=self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)][int(self.Prebleach_Frames):self.Nb_Frames]
            Residues = np.zeros(len(y))
            
            popt, pcov = curve_fit(single_exp, x, y, p0=None)

            xx = np.linspace(0,self.Nb_Frames,1000) #start, stop, num (number of samples to generate)
            yy = single_exp(xx, *popt)

            #Calculate Residues
            yyy = single_exp(x, *popt)
            Experimental_values = y.values.tolist() #conversion to array 
            Fitted_values = yyy.values.tolist() #conversion to array
            for i in range(0, len(Experimental_values)):
                Residues[i]=Experimental_values[i]-Fitted_values[i]
        
            #Subplot
            f, axarr = plt.subplots(2, sharex=True)
            axarr[0].plot(x,y,'x')
            axarr[0].plot(xx*self.SecondsPerFrame, yy,'r-')
            axarr[0].set_title(self.Name+', fitting to exponential')
            axarr[1].plot(x, Residues,'kx')
            axarr[1].plot([0,self.Nb_Frames], [0,0],'k--')
            axarr[1].set_title(self.Name+ ', residues of the fitting')

            print("Fitting results for #",Parser ,":\nMobile fraction=",round(popt[0],2), "and half time = ", round(self.Half_Time[Parser],2)) #print results of the fitting (a and b)
            print("Standard deviation of residues = ",round(np.std(Residues),3)) #print results of the fitting (a and b)

            plt.xlim(0, max(x))
            plt.show()
            
            
    def Plot_Data(self):
        """ Plot Data of an object from class Experiment """
        
        for Parser in range(0,len(self.txt_List)):
        #for Parser in range(0,1):
            x=self.Data['TIME']
            y=self.Data['#'+str(Parser)+'_FRAP_'+str(self.Subset)]
            z=self.Data['#'+str(Parser)+'_REF_'+str(self.Subset)]
            w=self.Data['#'+str(Parser)+'_BACK_'+str(self.Subset)]
            n=self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)]

            plt.figure(figsize=(20,6))
            
            plt.subplot(121)
            plt.plot(x,y,'x-',color = 'royalblue', label = 'FRAP (file#'+str(Parser)+')')
            plt.plot(x,z,'--',color = 'slategray', label = 'Reference (file#'+str(Parser)+')')
            plt.plot(x,w,'--',color = 'k', label = 'Background (file#'+str(Parser)+')')
            plt.xlabel(r'Time [s]')
            plt.ylabel(r'Intensity [AU] [0,255]')
            pylab.title(self.Name + ', File #' +str(Parser))
            plt.legend()
            
            plt.subplot(122)
            plt.plot(x,n,'x',color = 'seagreen', label = 'FRAP normalized(file#'+str(Parser)+')')
            plt.xlabel(r'Time [s]')
            plt.ylabel(r'Normalized Intensity [AU] [0,1]')
            pylab.title(self.Name+ ', Normalized, File #' +str(Parser))
            plt.legend()

            plt.show()

            
    def List(self):
        """ output ordered list of all txt files used in analysis """ 
        for i in range(0,len(self.txt_List)):
            print(i, ": ",self.txt_List[i],"\n")        

In [4]:
print("class was initiated")

class was initiated


# FRAPnoTime

In [5]:
class FRAPnoTime:
    """ For conditions without time-column in data-file """
    
                        #{0}  #{1}  #{2}     #{3}       #{4} 
    def __init__(self, Name, Path, Subset, Nb_Frames, Prebleach_Frames, SecondsPerFrame):

        self.Name = Name
        self.Path = Path
        self.Subset = Subset
        self.Nb_Frames = Nb_Frames
        self.Prebleach_Frames = Prebleach_Frames
        self.SecondsPerFrame = SecondsPerFrame

        self.txt_List = None
        self.Data = pd.DataFrame()
        self.Mean = []
        self.Std = []
        self.Mobile_Fraction_ave = None #Mobile fraction calculated by averaging all the data
        self.Mobile_Fraction = []
        self.Half_Time_ave = None #Half Time calculated by averaging all the data
        self.Half_Time = []
    
        print("Condition called '{0}' was successfully created".format(self.Name))   

        
    def __str__(self): #can call this with print(self)
        return """Condition:
 Name: {0} \n Path: {1} \n Subset: {2} \n Number of frames analyzed: {3} \n Number of pre-bleach frames: {4}
 Seconds per frame: {5}s""".format(self.Name, self.Path, self.Subset, self.Nb_Frames, self.Prebleach_Frames, self.SecondsPerFrame)

    
    def Extract_Data(self):
        """ Find .txt files and read them to extract data. Also compute normalizedFS data """
        
        print("Find data for ", self.Name)
        #PART 1 - Get txt_list                                                                      
        self.txt_List = sorted(Path(self.Path).glob('*' + self.Subset + '*.txt'))        
        #       - Get Data
        Parser = 0
        print("Extract data on",self.Nb_Frames, "frames and for",len(self.txt_List) ,"files")
        
        while Parser < len(self.txt_List):           
            #Read file and get Raw_Data
            with open(str(self.txt_List[Parser]), 'r') as file:                         
                Raw_Data= pd.read_csv(file, sep = '\t')                            
                Raw_Data.columns = ['FRAME', 'FRAP', 'REF', 'BACK']
                if Parser==0:
                    self.Data['FRAME'] = pd.Series(Raw_Data['FRAME'].iloc[:self.Nb_Frames])
            
        #PART 2 - Normalise FRAP Intensities
            Prebleach = Raw_Data[:int(self.Prebleach_Frames)]                    #Get pre-bleach data                                                   
           
            CorrPreREF = Prebleach['REF']-Prebleach['BACK']                      #Compute parameters
            Iref_pre = CorrPreREF.mean()
            CorrPreFRAP = Prebleach['FRAP']-Prebleach['BACK']
            Ifrap_pre = CorrPreFRAP.mean()
            
            Ifrap = Raw_Data['FRAP']
            Iref = Raw_Data['REF']
            Iback = Raw_Data['BACK']
            
            Inorm = (Iref_pre / (Iref - Iback)) * ((Ifrap - Iback) / Ifrap_pre)  #Double normalisation
                    #see FRAPAnalyser here: http://actinsim.uni.lu/eng/Downloads for more insights 
    
            Inorm_pre = 0                                                        #Full scaling starts
            for i in range(self.Prebleach_Frames):
                Inorm_pre = Inorm_pre + Inorm[i]
            Inorm_pre = Inorm_pre / self.Prebleach_Frames
            Inorm_FULLSCALE = (Inorm - Inorm[self.Prebleach_Frames])/(Inorm_pre - Inorm[self.Prebleach_Frames])
    
            Raw_Data['NormalizedFS'] = pd.Series(Inorm_FULLSCALE)
        
            #Add df to DATA
            self.Data['#'+str(Parser)+'_FRAP_'+str(self.Subset)] = pd.Series(Raw_Data['FRAP'].iloc[:self.Nb_Frames])
            self.Data['#'+str(Parser)+'_REF_'+str(self.Subset)] = pd.Series(Raw_Data['REF'].iloc[:self.Nb_Frames])
            self.Data['#'+str(Parser)+'_BACK_'+str(self.Subset)] = pd.Series(Raw_Data['BACK'].iloc[:self.Nb_Frames])
            self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)] = pd.Series(Raw_Data['NormalizedFS'].iloc[:self.Nb_Frames])
            
            Parser += 1
        
        print("Raw data extracted and normalised.")

        
    def Mean_and_Std(self):
        """ Compute Mean and Standard deviation """            
        
        tmp = [0]*len(self.txt_List) # creates array with y values for each time point (length txtList)

        for i in range(0,self.Nb_Frames):
            for Parser in range(0,len(self.txt_List)):
                tmp[Parser] = self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)][i]
            self.Mean.append(np.mean(tmp))
            self.Std.append(np.std(tmp))
        
        print("Mean and Sandard deviation were successfully computed.")

        
    def FitData(self):
        """ Fit all normalised recovery curves and plot fit with std """
    
        for Parser in range(0,len(self.txt_List)):
            x=(self.Data['FRAME'][0:self.Nb_Frames-int(self.Prebleach_Frames)]-1)*self.SecondsPerFrame
            y=self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)][int(self.Prebleach_Frames):self.Nb_Frames]
            Residues = np.zeros(len(y))
        
            popt, pcov = curve_fit(single_exp, x, y)
    
            #Store values
            self.Mobile_Fraction.append(popt[0])
            self.Half_Time.append(-1*np.log(0.5)/(popt[1]/self.SecondsPerFrame))
            
        print(str(len(self.txt_List)) + " data was fit as well as MF & HT computed.")

        

    def PlotFitWerror(self, outDir=None, single=False, color1 = 'black', color2 = 'forestgreen', color3  = 'peru', 
                     xAxis = r'time [s]', yAxis = 'norm. intensity'):
        """ Fit all normalised recovery curves and plot fit with std """
        
        #choosing single or double exponential fit as a model:
        if single == True:
            exponential = single_exp
            print("you are fitting with a single exponential")
        elif single == False:
            exponential = double_exp
            print("you are fitting with a double exponential")
        
        #Redo the fit:   
        y_mean_plus = []                                                             #upper bound
        for i in range(0,self.Nb_Frames):
            y_mean_plus.append(self.Mean[i] + self.Std[i])  
        y_mean_minus = []                                                            #lower bound 
        for i in range(0,self.Nb_Frames):
            y_mean_minus.append(self.Mean[i] - self.Std[i])

        x=(self.Data['FRAME'][0:self.Nb_Frames-int(self.Prebleach_Frames)]-1)*self.SecondsPerFrame
        y=self.Mean[int(self.Prebleach_Frames):self.Nb_Frames]
        popt, pcov = curve_fit(exponential, x, y, p0=None)
        
        xx = np.linspace(0,self.Nb_Frames,1000)
        yy = exponential(xx, *popt)

        xxx = ((self.Data['FRAME'] - self.Data['FRAME'][self.Prebleach_Frames]))*self.SecondsPerFrame      
        
        #output the fit:
        ax0 = plt.subplot2grid((3, 3), (0, 0), colspan=3, rowspan=4)                            #define geometry of the plot

        ax0.plot(xxx, self.Mean[:], 'b-', color = color1, label = 'Averaged FRAP recovery')  #Mean
        ax0.plot(xxx, y_mean_plus[:], "y--", color = color2)                                 #std+
        ax0.plot(xxx, y_mean_minus[:], "y--", color = color2)                                #std-
        ax0.plot(xx, yy,'k-', color = color3)                                                     #fit
        ax0.set_xlim(xxx[0], xxx[self.Nb_Frames-1])
        ax0.set_ylim([0, 1.2])
        ax0.set_title(self.Name)
        ax0.set_xlabel(xAxis)
        ax0.set_ylabel(yAxis)
        
        if outDir != None:
            plt.savefig(outDir + self.Name + ".svg")
            print("This plot was saved at: " + outDir)
        else:
            print("This plot was not saved")
 



    def PlotIndFit(self):
        """ Recalculate fits and plot individually """
    
        for Parser in range(0,len(self.txt_List)):
            x=(self.Data['FRAME'][0:self.Nb_Frames-int(self.Prebleach_Frames)]-1)*self.SecondsPerFrame
            y=self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)][int(self.Prebleach_Frames):self.Nb_Frames]
            Residues = np.zeros(len(y))
            
            popt, pcov = curve_fit(single_exp, x, y, p0=None)

            xx = np.linspace(0,self.Nb_Frames,1000) #start, stop, num (number of samples to generate)
            yy = single_exp(xx, *popt)

            #Calculate Residues
            yyy = single_exp(x, *popt)
            Experimental_values = y.values.tolist() #conversion to array 
            Fitted_values = yyy.values.tolist() #conversion to array
            for i in range(0, len(Experimental_values)):
                Residues[i]=Experimental_values[i]-Fitted_values[i]
        
            #Subplot
            f, axarr = plt.subplots(2, sharex=True)
            axarr[0].plot(x,y,'x')
            axarr[0].plot(xx*self.SecondsPerFrame, yy,'r-')
            axarr[0].set_title(self.Name+', fitting to exponential')
            axarr[1].plot(x, Residues,'kx')
            axarr[1].plot([0,self.Nb_Frames], [0,0],'k--')
            axarr[1].set_title(self.Name+ ', residues of the fitting')

            print("Fitting results for #",Parser ,":\nMobile fraction=",round(popt[0],2), "and half time = ", round(self.Half_Time[Parser],2)) #print results of the fitting (a and b)
            print("Standard deviation of residues = ",round(np.std(Residues),3)) #print results of the fitting (a and b)

            plt.xlim(0, max(x))
            plt.show()
            
            
    def Plot_Data(self):
        """ Plot Data of an object from class Experiment """
        
        for Parser in range(0,len(self.txt_List)):
        #for Parser in range(0,1):
            x=(self.Data['FRAME'])*self.SecondsPerFrame
            y=self.Data['#'+str(Parser)+'_FRAP_'+str(self.Subset)]
            z=self.Data['#'+str(Parser)+'_REF_'+str(self.Subset)]
            w=self.Data['#'+str(Parser)+'_BACK_'+str(self.Subset)]
            n=self.Data['#'+str(Parser)+'_NormalizedFS_'+str(self.Subset)]

            plt.figure(figsize=(20,6))
            
            plt.subplot(121)
            plt.plot(x,y,'x-',color = 'royalblue', label = 'FRAP (file#'+str(Parser)+')')
            plt.plot(x,z,'--',color = 'slategray', label = 'Reference (file#'+str(Parser)+')')
            plt.plot(x,w,'--',color = 'k', label = 'Background (file#'+str(Parser)+')')
            plt.xlabel(r'Time [s]')
            plt.ylabel(r'Intensity [AU] [0,255]')
            pylab.title(self.Name + ', File #' +str(Parser))
            plt.legend()
            
            plt.subplot(122)
            plt.plot(x,n,'x',color = 'seagreen', label = 'FRAP normalized(file#'+str(Parser)+')')
            plt.xlabel(r'Time [s]')
            plt.ylabel(r'Normalized Intensity [AU] [0,1]')
            pylab.title(self.Name+ ', Normalized, File #' +str(Parser))
            plt.legend()

            plt.show()

            
    def List(self):
        """ output ordered list of all txt files used in analysis """ 
        for i in range(0,len(self.txt_List)):
            print(i, ": ",self.txt_List[i],"\n") 